In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler

In [10]:
red = pd.read_csv("/Users/ellalee/Downloads/wine+quality/winequality-red.csv", sep = ';')
white = pd.read_csv("/Users/ellalee/Downloads/wine+quality/winequality-white.csv", sep = ';')

red['color'] = 'red'
white['color'] = 'white'

df = pd.concat([red, white], ignore_index=True)

both = df
df_array = [red, white, both]
results = []

In [12]:
def categorize_quality(quality):
    if quality < 7:
        return 'low'
    else:
        return 'high'

df['quality_category'] = df['quality'].apply(categorize_quality)
red['quality_category'] = red['quality'].apply(categorize_quality)
white['quality_category'] = white['quality'].apply(categorize_quality)

In [19]:
for wine in df_array:
    X = wine.drop(['quality', 'quality_category', 'color'], axis=1)
    y = wine['quality_category']

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, random_state=42, test_size=0.2)

    svm = SVC(random_state=42, kernel='rbf')
    svm.fit(X_train, y_train)

    y_pred = svm.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    results.append(report)

In [29]:
wine_names = ['red', 'white', 'red and white']
#print(len(results), len(wine_names))

for i, report in zip(wine_names, results):
    print("SVM for " + i + " dataset")
    print(f"accuracy: {report['accuracy']}" )
    print("---------------------")

SVM for red dataset
accuracy: 0.875
---------------------
SVM for white dataset
accuracy: 0.8214285714285714
---------------------
SVM for red and white dataset
accuracy: 0.8369230769230769
---------------------
